In [2]:
#Libraries
from dotenv import load_dotenv
import os

In [5]:
#Loading Environment Vars
load_dotenv()

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"] = "ConversationalAIBot"
os.environ["LANGCHAIN_TRACING_V2"] = "true"

In [6]:
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate

model = ChatGroq(
    temperature = 0.2,
    model_name = "deepseek-r1-distill-llama-70b",
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a Research assistant from arxiv"),
        ("human", "{question}")
    ]
)

question = "What does attention is all you need talk about?"

chain = prompt | model

In [7]:
print(chain.invoke({"question": question}).content)

<think>
Okay, so I'm trying to understand what the "Attention Is All You Need" paper is about. I remember hearing that it's a big deal in machine learning, especially with the Transformer model. But I'm not exactly sure what it says beyond that. Let me try to break it down.

First, the title suggests that attention mechanisms are the key focus here. I know that attention is a concept where the model focuses on different parts of the input when processing each part of the output. But how does that work exactly?

The paper is from 2017 by Vaswani et al., right? So it's introducing the Transformer model. I think before Transformers, people used RNNs or LSTMs for sequence tasks like translation. But RNNs have issues with long-term dependencies because they process data sequentially. That's where the Transformer comes in, I guess.

The Transformer uses self-attention, which allows the model to look at all parts of the input when generating each part of the output. That should help with capt

#### Loading the CSV file

In [8]:
import pandas as pd
from langchain_core.documents import Document

df = pd.read_csv("bitext-telco-llm-chatbot-training-dataset.csv")
df.head()

,instruction,intent,category,tags,response
0,there are charges on my phone bill that i do n...,dispute_invoice,BILLING,BCELQZ,If you have noticed discrepancies in your bill...
1,"my internet bill is incorrect, can you help me...",dispute_invoice,BILLING,BCILZ,If there is an issue with your bill and you wo...
2,my invoice is not corect challenge it,dispute_invoice,BILLING,BCELQZ,If you find any discrepancies in your invoice ...
3,"I don't recognize some charges on my bill, can...",dispute_invoice,BILLING,BCIL,If you have found charges on your bill that yo...
4,i have to dispute a fucking bill i need help,dispute_invoice,BILLING,BCQW,If you believe there is an error on your bill ...


In [ ]:
#Itering rows for organizing: Tried with classical CSV loader, this was efficient
conversation_documents = []

for index, row in df.iterrows():
    content = f""""Instruction: 
    {row['instruction']}
    Intent: {row['intent']}
    Category: {row['category']}
    Tags: {row['tags']}
    Response: {row['response']}"""

    conversation_documents.append(Document(page_content=content, metadata={"intent": row['intent']}))

In [10]:
print(conversation_documents[0])

page_content='"Instruction: 
    there are charges on my phone bill that i do not recogniae challenge it
    Intent: dispute_invoice
    Category: BILLING
    Tags: BCELQZ
    Response: If you have noticed discrepancies in your bill and wish to contest them, please adhere to the following steps:

1. Log in to your account on {{WEBSITE_URL}}.
2. Navigate to the {{INVOICE_SECTION}} section.
3. Select the bill you wish to dispute.
4. Click on the {{DISPUTE_INVOICE_OPTION}} to dispute the charge.
5. Fill in the required information and submit your dispute.

Our team will review your submission and get back to you within {{DAYS_NUMBER}} business days.' metadata={'intent': 'dispute_invoice'}


In [ ]:
#Indexing and Storing Faiss